In [1]:
import json

In [93]:
class Gates:
    
    def NOT(self, tup):
        try:
            x = tup[0]
        except:
            x = tup
        if x is 1:
            return 0
        return 1
       
    def OR(self, tup):
        x, y = tup
        if x == y and x == 0:
            return 0
        return 1
    
    def AND(self, tup):
        x, y = tup
        if x == y and x == 1:
            return 1
        return 0
    
    def XOR(self, tup):
        x, y = tup
        if x == y:
            return 0
        return 1
    
    def NOR(self, tup):
        return self.NOT(self.OR(tup))
    
    def NAND(self, tup):
        return self.NOT(self.AND(tup))
   
    def XNOR(self, tup):
        return self.NOT(self.XOR(tup))


In [100]:
def load(filename="json/f.smart.json"):
    with open(filename) as json_file:
        json_circuits = json.load(json_file)

    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
        circuit.printall()

class Circuit:
    def __init__(self,circuitDict):
        self.name = None
        self.alice = None
        self.bob = None
        self.gates = None
        self.out = None
        # processes json.
        self.processCircuit(circuitDict)

    def processCircuit(self,circuit):
        """
        Loads the json and adds the relevant contents into the
        object.
        """
#         for key in self.__dict__.keys():
        for key in circuit.keys():
            setattr(self,key,circuit[key])
        self.raw = circuit
            
    def compute(self,inputs):
        """
        Computes the gate operations and then returns the output.
        """
        # initiate gates
        gates = Gates()
        # creates stores.
        limit = max([max(self.out),max([i['id'] for i in self.gates])])
                    
        store = [0 for i in range(limit+1)]
        
        # store inputs
        for i in range(len(inputs)):
            store[i+1] = inputs[i]
        
        # iterate through the gate operations
        for gate in self.gates:
            # load logic gate
            logic = getattr(gates,gate['type'])
            # load inputs
            parameters = tuple([store[i] for i in gate['in']])
            # get output
            result = logic(parameters)
            # store result
            store[gate['id']] = result

        return [store[i] for i in self.out]
    
    def printall(self):
        """
        Helper function to handle printing as is specified in the
        coursework notes.
        """
        print(self.name)
        for alice in self.perms(len(self.alice)):
            if self.bob:
                for bob in self.perms(len(self.bob)):
                    inputs = alice + bob
                    pr = "Alice"+str(self.alice)+" = " 
                    for i in alice:
                        pr += str(i) + " "
                    pr += "  Bob"+str(self.bob)+" = "
                    for i in bob:
                        pr += str(i) + " "
                    pr += "  Outputs"+str(self.out)+" = "
                    for i in self.compute(inputs):
                        pr += str(i) + " "
                    print(pr)
            else:
                inputs = alice
                pr = "Alice"+str(self.alice)+" = " 
                for i in alice:
                    pr += str(i) + " "
                pr += "  Outputs"+str(self.out)+" = "
                for i in self.compute(inputs):
                    pr += str(i) + " "
                print(pr)
        print()
        
    @staticmethod
    def perms(n):
        """
        Helper function to generate permutations for binary integers
        based on the length n.
        """
        if not n:
            return
        entries = []
        for i in range(2**n):
            s = bin(i)[2:]
            s = "0" * (n-len(s)) + s
            ent = [int(i) for i in s]
            entries.append(ent)
        return entries
        
# load()


def loadall():
    import os
    folderpath = "json"
    for file in os.listdir(folderpath):
        filename = os.path.join(folderpath,file)
        with open(filename) as json_file:
            json_circuits = json.load(json_file)

        for json_circuit in json_circuits['circuits']:
            circuit = Circuit(json_circuit)
            circuit.printall()
loadall()

AND gate
Alice[1] = 0   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 0   Bob[2] = 1   Outputs[3] = 0 
Alice[1] = 1   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 1   Bob[2] = 1   Outputs[3] = 1 

OR gate
Alice[1] = 0   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 0   Bob[2] = 1   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 0   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 1   Outputs[3] = 1 

NOT gate
Alice[1] = 0   Outputs[2] = 1 
Alice[1] = 1   Outputs[2] = 0 

XOR gate
Alice[1] = 0   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 0   Bob[2] = 1   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 0   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 1   Outputs[3] = 0 

OR gate
Alice[1] = 0   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 0   Bob[2] = 1   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 0   Outputs[3] = 1 
Alice[1] = 1   Bob[2] = 1   Outputs[3] = 1 

NOR gate
Alice[1] = 0   Bob[2] = 0   Outputs[3] = 1 
Alice[1] = 0   Bob[2] = 1   Outputs[3] = 0 
Alice[1] = 1   Bob[2] = 0   Outputs[3] = 0 
Alice[1] = 1   Bob[2] = 1   Outputs[3] = 0 

